In [2]:
import numpy as np
import xarray as xr
import scipy
from scipy import stats
import scipy.special as sp
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from glob import glob
import os

In [3]:
cFlx = xr.open_mfdataset(sorted(glob('/Users/marki/Downloads/CERES_FluxByCldTyp-MON_Terra-Aqua-MODIS_Ed4.1_Subset_allsky_clearsky.nc')))

syn = xr.open_mfdataset(sorted(glob('/Users/marki/Downloads/CERES_SSF1deg-Month_Aqua-MODIS_Ed4.1_Subset_200207-202203.nc'))
#colors = {'Cu' : cm.Blues(.25), 'Sc' : cm.Blues(.5), 'St' : cm.Blues(.75), 'Ac' : cm.Purples(.25), 'As' : cm.Purples(.5), 'Ns' : cm.Purples(.75), 'Ci' : cm.Reds(.25), 'Cs' : cm.Reds(.5), 'Cb' : cm.Reds(.75),'all' : '.5'}

SyntaxError: unexpected EOF while parsing (<ipython-input-3-c4ce1e795110>, line 4)

In [ ]:
#
###Functions to calculate cloud fraction and albedo
#

def get_A_binned(ct,bin_width=.1):
    """
    Bin albedo values by cloud fraction for a given cloud type
    
    Parameters
    ----------
    ct : str
    Cloud type ('Cu','Sc','St','Ac','As','Ns','Ci','Cs', or 'Cb')
    
    bin_width : float
    Width of cloud fraction bins. Default is 0.1 (10%)
    
    Returns
    -------
    Cbinc : array
    Centers of cloud fraction bins
    
    A_binned : array-like
    List of arrays of albedo data for each cloud fraction bin
    """

    CF_ = np.ravel(dFlx[ct]['cldarea_cldtyp_mon']/100)
    Ac_ = np.ravel(dFlx[ct]['toa_albedo_cldtyp_mon'])
    As_ = np.ravel(cFlx['toa_sw_clr_mon']/cFlx['toa_solar_all_mon'])
    A_ = CF_*Ac_ + (1-CF_)*As_

    #Mask out invalid data (NaNs, land, ice)
    ocn = np.ravel(syn['aux_ocean_mon']) == 100
    nan = np.logical_or(np.isnan(CF_),np.isnan(A_))
    valid = np.logical_and(ocn,~nan)

    CF = CF_[valid]
    A = A_[valid]

    Cbinc = np.arange(bin_width/2,1,bin_width)
    A_binned = [(A[np.logical_and(CF>Cc-bin_width/2,CF<=Cc+bin_width/2)]) for Cc in Cbinc]
    
    return Cbinc, A_binned


In [ ]:
cldtypes = ['Cu','Sc','St','Ac','As','Ns','Ci','Cs','Cb']
dFlx = {} #Each cloud type aggregated separately
for cldtype in cldtypes:
    dFlx[cldtype] = xr.open_dataset('../%s_climo_mon.nc' % cldtype)

Important Line
``Cbinc, A_binned = get_A_binned(cldtype,bin_width=.1)``

In [ ]:
for cldtype in cldtypes:
    Cbinc, A_binned = get_A_binned(cldtype,bin_width=.1)
    print('----------------------------------------')
    print('Values for '+cldtype)
    print(Cbinc)
    print(A_binned)

In [1]:
np.shape(Cbinc)

NameError: name 'np' is not defined